In [1]:
pip install numpy pandas scikit-learn xgboost tensorflow lightgbm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1 -> 23.2.1
[notice] To update, run: C:\Users\Apurva\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, SimpleRNN, GRU, Dense
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [3]:
# Load your dataset
data = pd.read_csv('merged_file.csv')

In [4]:
data['date'] = pd.to_datetime(data['date'])
data.set_index('date', inplace=True)

In [5]:
# Check for remaining NaN values
nan_count = data.isna().sum().sum()
print(f"Total NaN values: {nan_count}")

Total NaN values: 151336


In [6]:
data.fillna(method='bfill', inplace=True)

In [7]:
# Check for remaining NaN values
nan_count = data.isna().sum().sum()
print(f"Total NaN values: {nan_count}")

Total NaN values: 0


In [8]:
# Split the dataset using fixed split
train_size = int(len(data) * 0.8)
train_data, test_data = data[:train_size], data[train_size:]

In [9]:
target_columns = ['Miscellaneous electric load for the South Wing', 'Miscellaneous electric load for the North Wing', 'Lighting load for the South Wing', 'Heating Ventilation and Air Conditioning load for the South Wing', 'Heating Ventilation and Air Conditioning load for the North Wing'] 

In [10]:
X_train, y_train = train_data.drop(target_columns, axis=1), train_data[target_columns]
X_test, y_test = test_data.drop(target_columns, axis=1), test_data[target_columns]

In [12]:
from keras.optimizers import Adam
from keras.layers import GRU

# Reshape the data for RNN-GRU (assuming a time window of 30 time steps)
time_steps = 30
X_train_rnn_gru = np.array([X_train.values[i:i + time_steps] for i in range(len(X_train) - time_steps)])
y_train_rnn_gru = y_train.iloc[time_steps:]
X_test_rnn_gru = np.array([X_test.values[i:i + time_steps] for i in range(len(X_test) - time_steps)])
y_test_rnn_gru = y_test.iloc[time_steps:]

rnn_gru_model = Sequential()
rnn_gru_model.add(GRU(50, input_shape=(time_steps, X_train_rnn_gru.shape[2])))  # Use GRU layer
rnn_gru_model.add(Dense(len(target_columns)))

# Define the custom optimizer with a lower learning rate (e.g., 0.001)
custom_optimizer = Adam(lr=0.001)
rnn_gru_model.compile(loss='mean_squared_error', optimizer=custom_optimizer)

rnn_gru_model.fit(X_train_rnn_gru, y_train_rnn_gru, epochs=50, batch_size=64, verbose=2)
rnn_gru_predictions = rnn_gru_model.predict(X_test_rnn_gru)
rnn_gru_rmse = mean_squared_error(y_test_rnn_gru, rnn_gru_predictions, squared=False)
print(f'RNN-GRU RMSE: {rnn_gru_rmse}')


Epoch 1/50
808/808 - 16s - loss: 145.6451 - 16s/epoch - 20ms/step
Epoch 2/50
808/808 - 14s - loss: 82.3841 - 14s/epoch - 18ms/step
Epoch 3/50
808/808 - 15s - loss: 79.7209 - 15s/epoch - 18ms/step
Epoch 4/50
808/808 - 15s - loss: 79.5887 - 15s/epoch - 18ms/step
Epoch 5/50
808/808 - 15s - loss: 79.4592 - 15s/epoch - 18ms/step
Epoch 6/50
808/808 - 14s - loss: 79.3137 - 14s/epoch - 17ms/step
Epoch 7/50
808/808 - 15s - loss: 79.1612 - 15s/epoch - 18ms/step
Epoch 8/50
808/808 - 15s - loss: 79.0114 - 15s/epoch - 19ms/step
Epoch 9/50
808/808 - 15s - loss: 78.8603 - 15s/epoch - 19ms/step
Epoch 10/50
808/808 - 15s - loss: 78.7492 - 15s/epoch - 18ms/step
Epoch 11/50
808/808 - 16s - loss: 78.6395 - 16s/epoch - 20ms/step
Epoch 12/50
808/808 - 16s - loss: 78.5419 - 16s/epoch - 20ms/step
Epoch 13/50
808/808 - 16s - loss: 78.4571 - 16s/epoch - 19ms/step
Epoch 14/50
808/808 - 15s - loss: 78.3895 - 15s/epoch - 19ms/step
Epoch 15/50
808/808 - 16s - loss: 78.3283 - 16s/epoch - 20ms/step
Epoch 16/50
808/80